In [16]:
import pandas as pd
from youtube_search import YoutubeSearch
from pytubefix import YouTube
import re
import os

### Load Scrapping Result

In [17]:
data = pd.read_csv('data/scrapping_result/lagu_daerah.csv', sep=',')

print(data.columns)
data.head()

Index(['No', 'Nama Lagu', 'Asal Daerah'], dtype='object')


,No,Nama Lagu,Asal Daerah
0,1,Bungong Jeumpa,Aceh
1,2,Jambo – Jambo,Aceh
2,3,Lembah Alas,Aceh
3,4,Aceh Lon Sayang,Aceh
4,5,Tawar Sedenge,Aceh


### Search and Download Music Function

In [10]:
def search_yt(query):
    results = YoutubeSearch(query, max_results=3).to_dict()

    for i in range(len(results)):
        results[i]['url'] = 'https://www.youtube.com' + results[i]['url_suffix']
        
    return results

def normalized_yt_title(title):
    text = title.lower()
    text = text.replace(' ', '_')
    text = re.sub(r'[^a-z0-9_]', '', text)
    text = re.sub(r'_{2,}', '_', text)
    
    return text

def dl_video(query):
    try:
        yt = YouTube(query, 'ANDROID')
        print(f'Downloading {yt.title}...')
        
        normalized_title = normalized_yt_title(yt.title)
        
        ys = yt.streams.get_audio_only()
        ys.download(mp3=True, output_path='datasets/songs', filename=normalized_title)
        
        return f'datasets/songs/{normalized_title}.mp3'
    except Exception as e:
        print(e)
        return None

In [11]:
dl_res = []

for index, row in data.iterrows():
    # Search for songs with the name and region keywords
    keyword = f"{row['Nama Lagu']} asal {row['Asal Daerah']}"
    searched_songs = search_yt(keyword)
    
    # Loop through the search results
    for song in searched_songs:
        try:
            duration = float(song.get('duration', 0))  # Ambil durasi dan default ke 0 jika tidak ada
            if duration <= 5:  # Filter berdasarkan durasi
                # Download the video using dl_video function
                path = dl_video(song['url'])
                
                # Append the result to dl_res list
                dl_res.append({
                    'title': song['title'],
                    'region': row['Asal Daerah'],
                    'keyword': keyword,
                    'duration': duration,
                    'url': song['url'],
                    'path': path
                })
            else:
                print(f"Duration of {song['title']} is too long")
        except Exception as e:
            print(f"Error processing song: {e}")



The ANDROID client requires PoToken to obtain functional streams, See more details at https://github.com/JuanBindez/pytubefix/pull/209
The ANDROID client requires PoToken to obtain functional streams, See more details at https://github.com/JuanBindez/pytubefix/pull/209


The ANDROID client requires PoToken to obtain functional streams, See more details at https://github.com/JuanBindez/pytubefix/pull/209
The ANDROID client requires PoToken to obtain functional streams, See more details at https://github.com/JuanBindez/pytubefix/pull/209


The ANDROID client requires PoToken to obtain functional streams, See more details at https://github.com/JuanBindez/pytubefix/pull/209
The ANDROID client requires PoToken to obtain functional streams, See more details at https://github.com/JuanBindez/pytubefix/pull/209


Duration of Jambo Jambo - Lagu Daerah Aceh (dengan Lirik) is too long


The ANDROID client requires PoToken to obtain functional streams, See more details at https://github.com/JuanBindez/pytubefix/pull/209
The ANDROID client requires PoToken to obtain functional streams, See more details at https://github.com/JuanBindez/pytubefix/pull/209


...

Duration of Waljinah - Caping Gunung' Langgam Campursari Sang Maestro is too long
Duration of Walang Kekek - Lagu Daerah Yogyakarta (Lirik & Terjemahan) [versi original] is too long
Duration of Walang Kekek [Versi Original]  - Lagu Daerah Yogyakarta - Jawa Tengah - Indonesia is too long


The ANDROID client requires PoToken to obtain functional streams, See more details at https://github.com/JuanBindez/pytubefix/pull/209
The ANDROID client requires PoToken to obtain functional streams, See more details at https://github.com/JuanBindez/pytubefix/pull/209


The ANDROID client requires PoToken to obtain functional streams, See more details at https://github.com/JuanBindez/pytubefix/pull/209
The ANDROID client requires PoToken to obtain functional streams, See more details at https://github.com/JuanBindez/pytubefix/pull/209


The ANDROID client requires PoToken to obtain functional streams, See more details at https://github.com/JuanBindez/pytubefix/pull/209
The ANDROID client requires PoToken to obtain functional streams, See more details at https://github.com/JuanBindez/pytubefix/pull/209


Duration of BUKAN BASIC KULINER TAPI NEKAT‼️USAHA MODAL PINJEM SODARA BERAWAL 3KG SEKARANG TERSEBAR PULUHAN TOKO is too long


### Save list of songs to csv

In [12]:
df = pd.DataFrame(dl_res)

df.to_csv('data/results.csv', sep=';', index=False)

### Convert to wav format

In [13]:
import os
import subprocess

os.makedirs('datasets/wav_songs', exist_ok=True)

def convert_to_wav(path):
    if path is None:
        print(f'File is not found: {path}')
        return None
    try:
        # path = path
        wav_path = path.replace('songs', 'wav_songs').replace('.mp3', '.wav')
        
        if os.path.exists(wav_path):
            return wav_path
        
        print(f'Converting {path} to:\n{wav_path}')
        subprocess.run(['ffmpeg', '-i', path, wav_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        return wav_path
    except Exception as e:
        print(e)
        return None

In [14]:
df['wav_path'] = df['path'].apply(convert_to_wav)

Converting datasets/songs/bungong_jeumpa_lirik_dan_terjemahan_lagu_daerah_aceh_dongeng_kita.mp3 to:
datasets/wav_songs/bungong_jeumpa_lirik_dan_terjemahan_lagu_daerah_aceh_dongeng_kita.wav
Converting datasets/songs/bungong_jeumpa_putri_ariani_cover_lagu_daerah_aceh.mp3 to:
datasets/wav_songs/bungong_jeumpa_putri_ariani_cover_lagu_daerah_aceh.wav
Converting datasets/songs/tari_bungong_jeumpa_aceh_rumah_belajar_mawinsya.mp3 to:
datasets/wav_songs/tari_bungong_jeumpa_aceh_rumah_belajar_mawinsya.wav
Converting datasets/songs/jambo_safira_amalia_official_music_video.mp3 to:
datasets/wav_songs/jambo_safira_amalia_official_music_video.wav
Converting datasets/songs/tarian_aceh_jambo_jambo.mp3 to:
datasets/wav_songs/tarian_aceh_jambo_jambo.wav
Converting datasets/songs/lembah_alas_lagu_daerah_aceh_dengan_lirik.mp3 to:
datasets/wav_songs/lembah_alas_lagu_daerah_aceh_dengan_lirik.wav
Converting datasets/songs/lembah_alas_lagu_daerah_aceh_enjoy_pesona_musik_indonesia.mp3 to:
datasets/wav_songs/lem

In [15]:
df.head()

df.to_csv('data/results_wav.csv', sep=';', index=False)